In [94]:
import numpy as np
from glob import glob
import os
from PIL import Image
import cv2
from tqdm import tqdm

images = glob("sybi/train/images/*.jpg")

sliced_images = list()

for image in tqdm(images, total=len(images)):
  fname = os.path.basename(image)
  label = "sybi/train/labels/"+fname[:-4]+".txt"
  image = np.array(Image.open(image))
  width, height, _ = image.shape

  mask = np.zeros((width, height), dtype=np.uint8)

  segmentations = list()
  bboxes = list()
  with open(label) as f:
    for line in f:
      coords = line.split(" ")
      x_coords = [int(float(coord) * width) for i, coord in enumerate(coords) if i % 2 != 0]
      y_coords = [int(float(coord) * height) for i, coord in enumerate(coords) if i % 2 == 0]
      del y_coords[0]

      segmentation = list(zip(x_coords, y_coords))
      segmentations.append(np.array(segmentation))
      bbox = [min(x_coords), min(y_coords), max(x_coords), max(y_coords)]
      bboxes.append(bbox)

      sliced_images.append(cv2.cvtColor(image[bbox[1]:bbox[3], bbox[0]:bbox[2], :],cv2.COLOR_BGR2RGB))

  mask = cv2.fillPoly(mask, segmentations, 1)

  masked_img1 = mask * image[:, :, 0]
  masked_img2 = mask * image[:, :, 1]
  masked_img3 = mask * image[:, :, 2]
  masked_img = cv2.merge((masked_img1, masked_img2, masked_img3))

100%|██████████| 5761/5761 [00:47<00:00, 122.49it/s]


In [96]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [106]:
train_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

class MyDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])
        if self.transform is not None:
            image = self.transform(image)

        return image, 0

batch_size = 64

train_dataset = MyDataset(sliced_images[:int(len(sliced_images) * 0.8)], transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = MyDataset(sliced_images[int(len(sliced_images) * 0.8):], transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size // 8, shuffle=False)

# FC AutoEncoder

In [107]:
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self, latent_dim=128):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # 3 * 64 * 64 -> 128 -> 64 -> 32 -> 16 -> 8 -> 4 -> 2 -> 1  ( 3 layers )
            nn.Linear(3 * 64 * 64, latent_dim*4), nn.ReLU(),
            nn.Linear(latent_dim*4, latent_dim*2), nn.ReLU(),
            nn.Linear(latent_dim*2, latent_dim), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, latent_dim*2), nn.ReLU(),
            nn.Linear(latent_dim*2, latent_dim*4), nn.ReLU(),
            nn.Linear(latent_dim*4, 3 * 64 * 64), nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(x.shape[0], 3, 64, 64)
        return x

In [136]:
import torch

epochs = 20

ae = Autoencoder(latent_dim=128).to('cuda')
optimizer = torch.optim.Adam(ae.parameters(), lr=3e-2)
criterion = nn.MSELoss()

for epoch in range(epochs):
  print(f"epoch : {epoch+1}")
  for data in tqdm(train_dataloader, total=len(train_dataloader)):
    images, _ = data
    images = images.to('cuda')
    reconstructed = ae(images)

    loss = criterion(reconstructed, images)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("train loss : ", loss.item())

  for data in tqdm(test_dataloader, total=len(test_dataloader)):
    images, _ = data
    images = images.to('cuda')
    reconstructed = ae(images)

    loss = criterion(reconstructed, images)

  print("test loss : ", loss.item())

epoch : 1


100%|██████████| 211/211 [00:06<00:00, 31.48it/s]


train loss :  0.19180947542190552


100%|██████████| 422/422 [00:01<00:00, 339.31it/s]


test loss :  0.12586964666843414
epoch : 2


...

test loss :  0.12586964666843414
epoch : 20


100%|██████████| 211/211 [00:06<00:00, 34.24it/s]


train loss :  0.1896684467792511


100%|██████████| 422/422 [00:01<00:00, 230.68it/s]

test loss :  0.12586964666843414


In [139]:
import time
sample = test_transform(Image.fromarray(sliced_images[1])).reshape(1, 3, 64, 64).to('cuda')
ae.eval()
start = time.time()
ae(sample)
print(time.time()-start)

0.0014302730560302734


# Conv AutoEncoder

In [121]:
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.linear = nn.Linear(128 * 16 * 16, latent_dim)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = x.view(-1, 128 * 16 * 16)
        x = self.linear(x)
        return x

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(latent_dim, 128 * 16 * 16)
        self.conv1 = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.conv2 = nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.ConvTranspose2d(32, 3, kernel_size=3, padding=1)

    def forward(self, z):
        x = self.linear(z).view(-1, 128, 16, 16)
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.interpolate(x, scale_factor=2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.interpolate(x, scale_factor=2)
        x = nn.functional.sigmoid(self.conv3(x))
        return x

class ConvAutoencoder(nn.Module):
  def __init__(self, latent_dim):
    super(ConvAutoencoder, self).__init__()
    self.encoder = Encoder(latent_dim)
    self.decoder = Decoder(latent_dim)

  def forward(self, x):
    z = self.encoder(x)
    reconstructed = self.decoder(z)
    return reconstructed

In [123]:
import torch

epochs = 20

cae = ConvAutoencoder(latent_dim=128).to('cuda')
optimizer = torch.optim.Adam(cae.parameters(), lr=3e-2)
criterion = nn.MSELoss()

for epoch in range(epochs):
  print(f"epoch : {epoch+1}")
  for data in tqdm(train_dataloader, total=len(train_dataloader)):
    images, _ = data
    images = images.to('cuda')
    reconstructed = cae(images)

    loss = criterion(reconstructed, images)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("train loss : ", loss.item())

  for data in tqdm(test_dataloader, total=len(test_dataloader)):
    images, _ = data
    images = images.to('cuda')
    reconstructed = cae(images)

    loss = criterion(reconstructed, images)

  print("test loss : ", loss.item())

epoch : 1


100%|██████████| 211/211 [00:06<00:00, 34.32it/s]


train loss :  0.17722299695014954


100%|██████████| 422/422 [00:01<00:00, 283.64it/s]


test loss :  0.09324374049901962
epoch : 2


...

test loss :  0.09324374049901962
epoch : 20


100%|██████████| 211/211 [00:05<00:00, 36.39it/s]


train loss :  0.1874171644449234


100%|██████████| 422/422 [00:01<00:00, 290.42it/s]

test loss :  0.09324374049901962


In [142]:
import time
sample = test_transform(Image.fromarray(sliced_images[1])).reshape(1, 3, 64, 64).to('cuda')
cae.eval()
start = time.time()
cae(sample)
print(time.time()-start)

0.015395879745483398


# UNet

In [128]:
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=3, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))
        self.outc = (OutConv(64, n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [129]:
import torch

epochs = 20

unet = UNet().to('cuda')
optimizer = torch.optim.Adam(unet.parameters(), lr=3e-2)
criterion = nn.MSELoss()

for epoch in range(epochs):
  print(f"epoch : {epoch+1}")
  for data in tqdm(train_dataloader, total=len(train_dataloader)):
    images, _ = data
    images = images.to('cuda')
    reconstructed = unet(images)

    loss = criterion(reconstructed, images)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("train loss : ", loss.item())

  for data in tqdm(test_dataloader, total=len(test_dataloader)):
    images, _ = data
    images = images.to('cuda')
    reconstructed = unet(images)

    loss = criterion(reconstructed, images)

  print("test loss : ", loss.item())

epoch : 1


100%|██████████| 211/211 [00:14<00:00, 14.11it/s]


train loss :  0.004628362134099007


100%|██████████| 422/422 [00:03<00:00, 124.66it/s]


test loss :  0.02566828392446041
epoch : 2


...

test loss :  0.01952999271452427
epoch : 20


100%|██████████| 211/211 [00:14<00:00, 14.40it/s]


train loss :  0.0004248737241141498


100%|██████████| 422/422 [00:03<00:00, 105.79it/s]

test loss :  0.01899695210158825


In [141]:
import time
sample = test_transform(Image.fromarray(sliced_images[1])).reshape(1, 3, 64, 64).to('cuda')
unet.eval()
start = time.time()
unet(sample)
print(time.time()-start)

0.058571577072143555
